# Notebook demonstrating the use of Timeseriesai for generating predictions
https://github.com/timeseriesAI/timeseriesAI

## The below is done using the tutorial data set (one state, one season) and is a very small dataset so the results are only intended for use on how to use this, not necessarily what the ML capability is

### uses env timeseriesai

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from tsai.all import *
from joblib import Parallel, delayed
from openavalancheproject.tsai_utilities import *
import numpy as np
import zarr

In [ ]:
#set_seed(42,False)

In [ ]:
data_root = '/media/scottcha/Data2/OAPMLData/'
data_root2 = '/media/scottcha/E1/Data/OAPMLData/'

ml_path = data_root + '/5.MLData/'
num_features = 774
interpolation = 1
label = 'Day1DangerAboveTreeline'
#file_label = 'ca_co_day1above_3h_large_rechunked'
days = '84d'
file_label = 'ca_co_day1above_3h_large'
#file_label = 'day1above'
#label = 'Day1DangerAboveTreeline'

In [ ]:
ml_path

In [ ]:
X = zarr.open(ml_path + 'X_' + file_label + '.zarr', mode='r')
#X2 = zarr.open(ml_path + 'co_ca_above.zarr', mode='r')

In [ ]:
#load the full X datafile which has both Train and Test concated
#X = np.load(ml_path + '/X_all_' + file_label + '.npy', mmap_mode='r')

In [ ]:
X.shape

In [ ]:
utils = TSAIUtilities(X, label)

In [ ]:
feature_names = pd.read_csv(ml_path + '/FeatureLabels_' + 'ca_co_day1above_3h_large' + '.csv')
feature_names = feature_names['0'].sort_values().values

In [ ]:
feature_names


In [ ]:
filtered_feature_list = utils.filter_features(set(feature_names), only_var=False)

In [ ]:
len(filtered_feature_list)

In [ ]:
len(pd.read_csv('TopFeatures.csv')['Feature'].values)

In [ ]:
filter_top_features = False
if filter_top_features:
    filtered_feature_list = pd.read_csv('TopFeatures.csv')['Feature'].values
    #filtered_feature_list = [x for x in filtered_feature_list if x in pd.read_csv('TopFeatures.csv')['Feature'].values]

In [ ]:
filtered_feature_indexes = [list(feature_names).index(x) for x in filtered_feature_list]

In [ ]:
len(filtered_feature_indexes)

In [ ]:
filtered_feature_indexes[-1]

In [ ]:
X.shape

In [ ]:
X_small_84 = X.get_orthogonal_selection((slice(None), filtered_feature_indexes, slice(1442-8*84, 1441)))

In [ ]:
#X_small_28 = X.get_orthogonal_selection((slice(None), filtered_feature_indexes, slice(1441-8*28, 1441)))

In [ ]:
X_small_84.shape

In [ ]:
X = X_small_84

In [ ]:
#feature distribution, some are gausian many are not, we'll use normalization 

In [ ]:
%%time
calc_train = False
if calc_train == True:
    feature_stds, feature_means = calc_std_and_mean(X, 0, 180000)

In [ ]:
%%time
calc_test = False
if calc_test == True:
    feature_stds_test, feature_means_test = calc_std_and_mean(X, 180000, X.shape[0])

In [ ]:
%%time
calc_min_train = False
if calc_min_train == True:
    feature_mins, feature_maxs = utils.calc_min_max(X, 0, 180000)

In [ ]:
%%time
calc_min_test = False
if calc_min_test == True:
    feature_mins_test, feature_maxs_test = calc_min_max(X, 180000, X.shape[0])

In [ ]:
#pd.Series(feature_stds).to_csv(data_root2 +  'std_ca_co_day1above_3h_large_84d.csv', index=False)
#pd.Series(feature_means).to_csv(data_root2 + 'mean_ca_co_day1above_3h_large_84d.csv', index=False)
#pd.Series(feature_stds_test).to_csv(data_root2 +  'std_test_ca_co_day1above_3h_large_84d.csv', index=False)
#pd.Series(feature_means_test).to_csv(data_root2 + 'mean_test_ca_co_day1above_3h_large_84d.csv', index=False)

In [ ]:
#feature_stds = pd.read_csv(data_root2 + '/std_' + file_label + '_' + days + '.csv' )['0'].values
#feature_means = pd.read_csv(data_root2 + '/mean_' + file_label + '_' + days + '.csv' )['0'].values
#feature_stds_test = pd.read_csv(data_root2 + '/std_test_' + file_label + '_' + days + '.csv' )['0'].values
#feature_means_test = pd.read_csv(data_root2 + '/mean_test_' + file_label + '_' + days + '.csv' )['0'].values

In [ ]:
#pd.Series(feature_mins).to_csv(data_root2 +  'min_ca_co_day1above_3h_large_84d.csv', index=False)
#pd.Series(feature_maxs).to_csv(data_root2 + 'max_ca_co_day1above_3h_large_84d.csv', index=False)
#pd.Series(feature_mins_test).to_csv(data_root2 +  'min_test_ca_co_day1above_3h_large_84d.csv', index=False)
#pd.Series(feature_maxs_test).to_csv(data_root2 + 'max_test_ca_co_day1above_3h_large_84d.csv', index=False)

In [ ]:

feature_mins = pd.read_csv(data_root2 + '/min_' + 'ca_co_day1above_3h_large' + '_' + days + '.csv' )['0'].values
feature_maxs = pd.read_csv(data_root2 + '/max_' + 'ca_co_day1above_3h_large' + '_' + days + '.csv' )['0'].values

#feature_mins_test = pd.read_csv(data_root2 + '/min_test_' + 'ca_co_day1above_3h_large' + '_' + days + '.csv' )['0'].values
#feature_maxs_test = pd.read_csv(data_root2 + '/max_test_' + 'ca_co_day1above_3h_large' + '_' + days + '.csv' )['0'].values

In [ ]:
#read in the corresponding label files and concat them
#can get the right values here based on the contents of hte ml_path directory

#num_train_files = 80
num_train_files = 60
file_list = []
for i in range(num_train_files):
    file_list.append(pd.read_parquet(ml_path + '/y_train_batch_' + str(i) + '_' + file_label + '.parquet'))

    
num_test_files = 12
#num_test_files = 4
for i in range(num_test_files):
    file_list.append(pd.read_parquet(ml_path + '/y_test_batch_' + str(i + num_train_files) + '_' + file_label + '.parquet'))
    #file_list.append(pd.read_parquet(ml_path + '/y_test_batch_' + str(i) + '_' + file_label + '.parquet'))
    
y_df = pd.concat(file_list).reset_index(drop=True)

In [ ]:
utils = TSAIUtilities(X, 'Day1DangerAboveTreeline')

In [ ]:
#utils.augment_labels_with_trends(data_root + 'CleanedForecastsNWAC_CAIC_UAC_CAC.V1.2013-2021.csv', y_df)

In [ ]:
labels = y_df['Day1DangerAboveTreeline'].unique()
labels

In [ ]:
y_df = y_df[y_df['Day1DangerAboveTreeline'].notna()].copy()

In [ ]:
#augmenting lables removes some so we need to ensure X has the same index
X = X[y_df.index]

In [ ]:
y_df.reset_index(inplace=True, drop=True)

In [ ]:
y_df['season'].value_counts()

In [ ]:
y_df['Day1DangerAboveTreeline'].value_counts()

In [ ]:
y, cat_dict = utils.get_y_as_cat(y_df)
cat_dict

In [ ]:
splits2 = (L(list(y_df[y_df['season']!='20-21'].index.values)).shuffle(), L(list(y_df[y_df['season']=='20-21'].index.values)).shuffle())

In [ ]:
splits, dls = utils.create_dls(X, y, feature_mins=feature_mins[filtered_feature_indexes], feature_maxs=feature_maxs[filtered_feature_indexes], sample_frac=1, splits=splits2)
splits

In [ ]:
#check both the train and test sets to ensure that there aren't full rows with all nulls

In [ ]:
%%time
#num_workers:12, 7.5s
#num_workers:6, 7.75s
#3, 7.56s
#1, 7.6
dls.show_batch()

In [ ]:
dls.valid.show_batch()

In [ ]:
dls.c

In [ ]:
dls.vars

In [ ]:
splits, dls = utils.create_dls(X, y, feature_mins=np.asarray(feature_mins[filtered_feature_indexes]), 
                                   feature_maxs=np.asarray(feature_maxs[filtered_feature_indexes]), 
                                   splits=splits, sample_frac=1)

In [ ]:
#Trial 183 finished with value: 0.6043333333333333 and parameters: 
#{'model': 'LSTM', 'hidden_size': 100, 'lstm_n_layers': 1, 'rnn_dropout': 0.8, 
#'bidirectional': True, 'lstm_fc_dropout': 0.2, 'lstm_cell_type': 'LSTM', 
#'loss_func': 'None', 'learning_rate': 0.0076197040198174755, 'learner_callback': 'None', 'optimizer': 'AdamWD'}
model = LSTMPlus(dls.vars, dls.c, hidden_size=100, n_layers=1, rnn_dropout=.8, bidirectional=True, 
                 fc_dropout=.2)
model.load_state_dict(torch.load("./models/" + 'lstm' + '.pth'))
learn = Learner(dls, model, loss_func=None, opt_func=partial(Adam, decouple_wd=True),  metrics=accuracy, cbs=None)   

In [ ]:
#{'model': 'LSTM', 'hidden_size': 150, 'lstm_n_layers': 3, 
#'rnn_dropout': 0.8, 'bidirectional': False, 'lstm_fc_dropout': 0.6000000000000001, 
#'lstm_cell_type': 'GRU', 'loss_func': 'None', 'learning_rate': 0.001038746221962459,
#'learner_callback': 'CutMix1d', 'optimizer': 'AdamWD'}
model = GRUPlus(dls.vars, dls.c, hidden_size=150, n_layers=3, rnn_dropout=.8, bidirectional=False, 
                fc_dropout=.6)
model.load_state_dict(torch.load("./models/" + 'gru' + '.pth'))
learn = Learner(dls, model, loss_func=None, opt_func=partial(Adam, decouple_wd=True),  metrics=accuracy, cbs=CutMix1d())   

In [ ]:
learn.fit(1, lr=.0001)

In [ ]:
learn.fit(200, lr=.001, cbs=[EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=5), SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=.001, fname='lstm')])

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
conf_matrix = interp.plot_confusion_matrix(normalize=True)
conf_matrix = interp.plot_confusion_matrix(normalize=False)

In [ ]:

interp = ClassificationInterpretation.from_learner(learn)
conf_matrix = interp.plot_confusion_matrix(normalize=True)
conf_matrix = interp.plot_confusion_matrix(normalize=False)

In [ ]:
X[splits[0]].shape

In [ ]:
??learn.feature_importance

In [ ]:
model.load_state_dict(torch.load("./models/" + 'gru' + '.pth'))

In [ ]:
list(feature_names)

In [ ]:
X[splits[1]].shape[1]

In [ ]:
len(feature_names[filtered_feature_indexes])

In [ ]:
#feature_names = feature_names[filtered_feature_indexes]

In [ ]:
len(list(feature_names))

In [ ]:

df_importance = learn.feature_importance(X[splits[1]], y[splits[1]], feature_names=list(feature_names[filtered_feature_indexes]))

In [ ]:
df_importance[:40]

In [ ]:
df_importance[-40:]

In [ ]:
df_importance[df_importance['accuracy_change'] > 0].to_csv('TopFeatures.csv')

In [ ]:
model = InceptionTimePlus(dls.vars, dls.c)
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=None, 
                cbs=[ShowGraphCallback2(), 
                     PredictionDynamics(), 
                     SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=.001, fname='default'), 
                     EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=5)]) 
#learn.lr_find()
learn.fit(50, lr=1e-5)

In [ ]:
splits, dls = utils.create_dls(X, y, feature_mins=np.asarray(feature_mins[filtered_feature_indexes]), 
                                   feature_maxs=np.asarray(feature_maxs[filtered_feature_indexes]), 
                                   splits=splits, sample_frac=1)

In [ ]:
#'model': 'InceptionTimePlus', 'itp_num_filters': 64, 'itp_depth': 6, 'itp_dropout_rate': 0.2, 'itp_ks': 50, 'loss_func': 'LabelSmoothingCrossEntropyFlat', 'learning_rate': 0.00010360777693535245, 'learner_callback': 'IntraClassCutMix1d', 'optimizer': 'AdamNoWD'
model = InceptionTimePlus(dls.vars, dls.c, nf=64, depth=6, ks=50, conv_dropout=.2)
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=LabelSmoothingCrossEntropyFlat(), 
                opt_func=partial(Adam, decouple_wd=False), 
                cbs=[ShowGraphCallback2(), 
                     PredictionDynamics(),
                     CutMix1d(),
                     SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=.001, fname='tuned'), 
                     EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=5)]) 
learn.fit_one_cycle(50, lr_max=1e-4)

In [ ]:
#above here

In [ ]:
model = InceptionTimePlus(dls.vars, dls.c)
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=None, cbs=[ShowGraphCallback2(), PredictionDynamics()]) 
learn.fit(20, lr=1e-2)

In [ ]:
model = InceptionTimePlus(dls.vars, dls.c, depth=8)
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=None, cbs=[ShowGraphCallback2(), PredictionDynamics()]) 
learn.fit(20, lr=1e-2)

In [ ]:
model = InceptionTimePlus(dls.vars, dls.c, depth=12)
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=None, cbs=[ShowGraphCallback2(), PredictionDynamics()]) 
learn.fit(20, lr=1e-2)

In [ ]:
model = InceptionTimePlus(dls.vars, dls.c, depth=3)
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=None, cbs=[ShowGraphCallback2(), PredictionDynamics()]) 
learn.fit(20, lr=1e-2)

In [ ]:
model = InceptionTimePlus(dls.vars, dls.c, depth=1)
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=None, cbs=[ShowGraphCallback2(), PredictionDynamics()]) 
learn.fit(40, lr=1e-2)

In [ ]:
model = InceptionTimePlus(dls.vars, dls.c, depth=2)
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=None, cbs=[ShowGraphCallback2(), PredictionDynamics()]) 
learn.fit(40, lr=1e-2)

In [ ]:
model = InceptionTimePlus(dls.vars, dls.c, depth=2)
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=None, opt_func = partial(Adam, decouple_wd=False), cbs=[ShowGraphCallback2(), PredictionDynamics()]) 
learn.fit(40, lr=1e-2)

In [ ]:
model = InceptionTimePlus(dls.vars, dls.c, depth=6)
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=None, opt_func = partial(Adam, decouple_wd=False), cbs=[ShowGraphCallback2(), PredictionDynamics()]) 
learn.fit(100, lr=1e-2)

In [ ]:
model = InceptionTimePlus(dls.vars, dls.c, depth=6)
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=None, opt_func = partial(Adam, decouple_wd=False, eps=.001), cbs=[ShowGraphCallback2(), PredictionDynamics()]) 
learn.fit(100, lr=1e-2)

In [ ]:
model = InceptionTimePlus(dls.vars, dls.c, depth=6)
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=None, opt_func = partial(Adam, decouple_wd=False), cbs=[ShowGraphCallback2(), PredictionDynamics()]) 
learn.fit(100, lr=1e-4)

# Lets try and train a model, in the tutorial case we are using an extremely small dataset so no conclusions should be drawn from these results they are for illustrative purposes only

In [ ]:
#top values from model zoo
#Trial 289 finished with value: 0.617 and parameters: {'num_days': 28, 'batch_size': 8, 'model': 'InceptionTimePlus', 'itp_num_filters': 32, 'itp_depth': 6, 'itp_dropout_rate': 0.0, 'loss_func': 'LabelSmoothingCrossEntropyFlat', 'learning_rate': 0.0024792334740007253}

#Trial 454 finished with value: 0.616 and parameters: {'num_days': 28, 'batch_size': 8, 'model': 'InceptionTimePlus', 'itp_num_filters': 32, 'itp_depth': 9, 'itp_dropout_rate': 0.0, 'loss_func': 'None', 'learning_rate': 0.0024792334740007253}

#Trial 161 finished with value: 0.609 and parameters: {'num_days': 84, 'batch_size': 8, 'model': 'InceptionTimePlus', 'itp_num_filters': 32, 'itp_depth': 9, 'itp_dropout_rate': 0.0, 'loss_func': 'None', 'learning_rate': 0.0006723398746247245}

#Trial 26 finished with value: 0.611 and parameters: {'num_days': 84, 'batch_size': 8, 'model': 'InceptionTimePlus', 'itp_num_filters': 32, 'itp_depth': 6, 'itp_dropout_rate': 0.2, 'loss_func': 'None', 'learning_rate': 1.339568695002037e-05}

#Trial 352 finished with value: 0.604 and parameters: {'num_days': 28, 'batch_size': 64, 'model': 'InceptionTimePlus', 'itp_num_filters': 64, 'itp_depth': 6, 'itp_dropout_rate': 0.0, 'loss_func': 'None', 'learning_rate': 0.0024792334740007253}

#good learning curve
#Trial 268 finished with value: 0.565 and parameters: {'num_days': 28, 'batch_size': 8, 'model': 'TCN', 'TCN_fc_dropout': 0.6000000000000001, 'TCN_conv_dropout': 0.0, 'loss_func': 'LabelSmoothingCrossEntropy', 'learning_rate': 1.339568695002037e-05}

In [ ]:
model = InceptionTimePlus(dls.vars, dls.c,  conv_dropout=.2, depth=6, nf=32)
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=LabelSmoothingCrossEntropyFlat(), cbs=[ShowGraphCallback2(), PredictionDynamics(),SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=.001)] )#.to_fp16()

In [ ]:
learn.fit(200, lr=1e-5, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=15))

In [ ]:
model = InceptionTimePlus(dls.vars, dls.c,  conv_dropout=.2, depth=6, nf=32)
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=LabelSmoothingCrossEntropyFlat(), cbs=[ShowGraphCallback2(), PredictionDynamics(),SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=.001), CutMix1D()] )#.to_fp16()

In [ ]:
learn.fit(200, lr=1e-5, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=15))

In [ ]:
model = InceptionTimePlus(dls.vars, dls.c,  conv_dropout=.2, depth=6, nf=32)
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=LabelSmoothingCrossEntropyFlat(), cbs=[ShowGraphCallback2(), PredictionDynamics(),SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=.001), CutMix1D()] ).to_fp16()

In [ ]:
learn.fit(200, lr=1e-5, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=5))

In [ ]:
model = TCN(dls.vars, dls.c, conv_dropout=0, fc_dropout=.6)
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=LabelSmoothingCrossEntropyFlat(), cbs=[ShowGraphCallback2(), PredictionDynamics(),SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=.001), CutMix1D()] )#.to_fp16()

In [ ]:
learn.fit(200, lr=1e-5, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=15))

In [ ]:
splits, dls = create_dls()

In [ ]:
#{'num_days': 112, 'batch_size': 8, 'model': 'XCM', 'window_perc': 0.2, 'xcm_nf': 192, 'xcm_fc_dropout': 0.8, 'loss_func': 'LabelSmoothingCrossEntropyFlat', 'learning_rate': 0.0002725629442830844}  
model = XCM(dls.vars, dls.c, dls.len, window_perc=.2, fc_dropout=.8, nf=192)
learn = Learner(dls, model, metrics=accuracy, loss_func=LabelSmoothingCrossEntropyFlat(), cbs=[ShowGraphCallback2(), SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=.001)])


In [ ]:
??LabelSmoothingCrossEntropyFlat

In [ ]:
?TST

In [ ]:
#Try TST
#{'d_ff': 512, 'd_k': 64, 'd_model': 128, 'd_v': 64, 'dropout': 0.7, 'fc_dropout': 0.1, 'loss': 'LabelSmoothingCrossEntropyFlat', 'n_heads': 16, 'n_layers': 4}
model = TSTPlus(dls.vars, dls.c, dls.len, d_ff=512, d_k=64, d_model=128, d_v=64, dropout=.7, fc_dropout=.1, n_heads=16, n_layers=4)
learn = Learner(dls, model, loss_func=LabelSmoothingCrossEntropyFlat(), metrics=[accuracy, BalancedAccuracy()],  cbs=[ShowGraphCallback2(), PredictionDynamics(), SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=.001)]).to_fp16()

In [ ]:
model = MultiTSTPlus(dls.vars, dls.c, dls.len)
learn = Learner(dls, model, loss_func=LabelSmoothingCrossEntropyFlat(), metrics=accuracy,  cbs=ShowGraphCallback2())

In [ ]:
#try rocket
model = build_ts_model(ROCKET, dls=dls) 
X_train, y_train = create_rocket_features(dls.train, model)
X_valid, y_valid = create_rocket_features(dls.valid, model)
X_train.shape, X_valid.shape

In [ ]:
from sklearn.linear_model import RidgeClassifierCV
ridge = RidgeClassifierCV(alphas=np.logspace(-8, 8, 17), normalize=True)
ridge.fit(X_train, y_train)
print(f'alpha: {ridge.alpha_:.2E}  train: {ridge.score(X_train, y_train):.5f}  valid: {ridge.score(X_valid, y_valid):.5f}')

In [ ]:
#model = InceptionTimePlus(dls.vars, dls.c)
model = InceptionTimePlus(dls.vars, dls.c, bottleneck=True, coord=False, ks=5, conv_dropout=.8, depth=6, nf=32)
learn = Learner(dls, model, metrics=accuracy, loss_func=LabelSmoothingCrossEntropy(), cbs=ShowGraphCallback2()).to_fp16()

In [ ]:
learn.lr_find()

In [ ]:
#XCM
learn.fit(200, lr=1e-2, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=5))

In [ ]:
#TSTPlus
learn.fit(20, lr=1e-4, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=5))

In [ ]:
#TST
learn.fit(20, lr=1e-4, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=5))

In [ ]:
learn.fit_one_cycle(200, lr_max=1e-4, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=25))

In [ ]:
#A100
learn.fit_one_cycle(25, lr_max=1e-3, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=3))

In [ ]:
#fp32
learn.fit_one_cycle(25, lr_max=1e-3, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=3))

### Results


In [ ]:
list(splits_2[1].sorted())

In [ ]:
learn.feature_importance(X.get_orthogonal_selection((list(splits_2[1].sorted()), slice(None), slice(None))), list(y[splits_2[1].sorted()]))

In [ ]:
??learn.feature_importance

In [ ]:
pd.Series(feature_names)[923]

In [ ]:
learners = {}
#learners['ITP'] = []
learners['TST'] = []
#learners['XCM'] = []

In [ ]:
learners.keys()

In [ ]:
learn.

In [ ]:
model.load_state_dict(torch.load("./models/" + 'model' + '.pth'))
learn = Learner(dls, model, metrics=accuracy, loss_func=LabelSmoothingCrossEntropyFlat(), cbs=[ShowGraphCallback2(), SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=.001)])

In [ ]:
splits, dls = utils.create_dls(X, y, feature_mins=feature_mins[filtered_feature_indexes], feature_maxs=feature_maxs[filtered_feature_indexes], sample_frac=.5)

In [ ]:
model = InceptionTimePlus(dls.vars, dls.c)#, nf=64, depth=6, ks=50)
from tsai.inference import load_learner
model.load_state_dict(torch.load("./models/" + 'default' + '.pth'))
#learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=LabelSmoothingCrossEntropyFlat(), cbs=[ShowGraphCallback2(), PredictionDynamics(),SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=.001)] )#.to_fp16()
learn = Learner(dls, model, metrics=[accuracy, BalancedAccuracy()], loss_func=None, 
                cbs=[ShowGraphCallback2(), 
                     PredictionDynamics(), 
                     SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=.001, fname='default'), 
                     EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=5)]) 

In [ ]:
probas, target, preds = learn.get_preds(dl=dls.valid, with_input=False, with_loss=False, with_decoded=True, act=None, reorder=False)

In [ ]:
target

In [ ]:
preds

In [ ]:
skm.accuracy_score(target, preds)

In [ ]:
#d,t = flatten_check(interp.decoded, interp.targs)
#print(str(skm.accuracy_score(t, d)))
interp = ClassificationInterpretation.from_learner(learn)
conf_matrix = interp.plot_confusion_matrix(normalize=False)
conf_matrix = interp.plot_confusion_matrix(normalize=True)

In [ ]:
interp.most_confused()

In [ ]:
len(np.where(abs(target-preds) >= 9)[0])

In [ ]:
y_df.iloc[np.where(abs(target-preds) == 10)[0]]['parsed_date'].dt.month.value_counts()

In [ ]:
#valid mismatches
y_df.iloc[splits[1][np.where(abs(target-preds) == 10)[0]]][['Day1Date', 'UnifiedRegion']].groupby(['Day1Date', 'UnifiedRegion']).size()

In [ ]:
#train mismatches
pd.set_option('display.max_rows', 1500)
y_df.iloc[splits[1][np.where(abs(target-preds) == 3)[0]]][['Day1Date', 'UnifiedRegion']].groupby(['Day1Date','UnifiedRegion']).size()


In [ ]:
y_df.iloc[splits[1][np.where(abs(target-preds) == 3)[0]]]['Day1Date'].value_counts()

In [ ]:
t = y_df[ y_df['Day1Date'] == 20201217.0 ]
t = t[t['UnifiedRegion'] == 'Northwest Coastal']
t.sort_values(['latitude', 'longitude'])


In [ ]:
t = y_df[ y_df['Day1Date'] == 20201218.0 ]
t = t[t['UnifiedRegion'] == 'Northwest Coastal']
t.sort_values(['latitude', 'longitude'])



In [ ]:
filtered_feature_list

In [ ]:
def plot_features(x, x2=None):
    for i in range(x.shape[0]):
        print(filtered_feature_list[i])
        pd.Series(x[i]).plot()
        plt.show()
        if x2 is not None:
            pd.Series(x2[i]).plot()
            plt.show()
            

In [ ]:
plot_features(X[210237], X[189293])

In [ ]:
X[210237][list(filtered_feature_list).index('SNOD_surface_avg')]

In [ ]:
t = y_df[ y_df['Day1Date'] == 20201210.0 ]
t = t[t['UnifiedRegion'] == 'Purcells']
t['Day1DangerAboveTreeline'].head()

In [ ]:
len(target)

In [ ]:
180000-179968

In [ ]:
splits_2

In [ ]:
np.where(abs(target-preds) == 3)[0][8:16]

In [ ]:
y_df.iloc[179944]['Day1DangerAboveTreeline']

In [ ]:
target[48]

In [ ]:
preds[48]

In [ ]:
splits_2[0][8]

In [ ]:
pd.set_option('display.max_rows', 1500)
y_df.iloc[splits[0][48]]

In [ ]:
#predict high when forecast is low 
for i in np.where(abs(target-preds) >= 3)[0][0:8]:
    print(i)
    print('split ' + str(splits[1][i]))
    print('y ' + str(y[splits[1][i]]))
    print(y_df.iloc[splits[1][i]]['Day1DangerAboveTreeline'])
    pd.DataFrame(X[splits[1]][i]).T.plot(legend=False)
    plt.show()

In [ ]:
#sample of correct preds
for i in np.where(abs(target-preds) == 0)[0][:8]:
    print(i)
    print('split ' + str(splits[1][i]))
    print('y ' + str(y[splits[1][i]]))
    print(y_df.iloc[splits[1][i]]['Day1DangerAboveTreeline'])
    pd.DataFrame(X[splits[1]][i]).T.plot(legend=False)
    plt.show()

In [ ]:
pd.Series(feature_names)

In [ ]:
y_df.iloc[1]

In [ ]:
X[0,19]

In [ ]:
fil

In [ ]:
X[187699].shape

In [ ]:
for i in range(len(filtered_feature_indexes)):
    
    low = i
    high = i+1
               
    print('Incorrect Low Pred')

    pd.DataFrame(X[187699][low:high].T, columns=filtered_feature_list[low:high]).plot(legend=False)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

    print('Correct Low Pred')
    pd.DataFrame(X[202437][low:high].T, columns=filtered_feature_list[low:high]).plot(legend=False)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()


In [ ]:
pd.DataFrame(X[1001][low:high].T, columns=feature_names[filtered_feature_indexes][low:high]).describe()

In [ ]:
incorrect_num = 0
for i in range(X.shape[0]):
    x = np.copy(X[i][low:high])
    x = np.nan_to_num(x)
    np.mod(x, 1, out=x)
    mask = (x == 0)
    if np.all(mask):
        incorrect_num += 1

In [ ]:
incorrect_num

In [ ]:
X.shape

In [ ]:
pd.DataFrame(X[144769][low:high].T, columns=feature_names[filtered_feature_indexes][low:high]).describe()

In [ ]:
pd.DataFrame(X[0][low:high].T, columns=feature_names[filtered_feature_indexes][low:high])

In [ ]:
pd.Series(feature_names)

In [ ]:
tmp_feature_names = feature_names[filtered_feature_indexes]

In [ ]:
del feature_names

In [ ]:
?learn.feature_importance

In [ ]:
len(tmp_feature_names)

In [ ]:
X.shape

In [ ]:
learn.feature_importance(X[:256], y[:256])
#learn.feature_importance(X[np.where(abs(target-preds) >= 9)], y[np.where(abs(target-preds) >= 9)])

In [ ]:
tmp_feature_names[47]

In [ ]:
learn.feature_importance(X[np.where(abs(target-preds) == 0)[0][:1000]], y[np.where(abs(target-preds) == 0)[0][:1000]])

In [ ]:
top_features = [31, 42, 20, 1, 19, 51, 36, 24, 18, 4, 5, 32, 53, 16, 12, 26, 30, 17, 0, 10, 48, 55, 23, 29, 8, 14, 35, 9, 43, 7, 3, 52, 33]
pd.Series(feature_names[filtered_feature_indexes][top_features]).to_csv('top_features.csv', index=False)

In [ ]:
#most important features for incorrect
tmp_feature_names[[13, 42, 1, 20, 9]]

In [ ]:
#most important features for correct
tmp_feature_names[[13, 42, 20, 1, 19]]

In [ ]:
??learn.feature_importance

In [ ]:
len(y[np.where(abs(target-preds) == 3)])

In [ ]:
len(feature_names[filtered_feature_indexes])

In [ ]:
X[np.where(abs(target-preds) == 3)].shape

In [ ]:
y_df.iloc[splits_2[1][1]]['Day1DangerAboveTreeline']

In [ ]:
interp.targs - np.argmax(interp.preds, axis=1)

In [ ]:
y_df.shape

In [ ]:
y_df.iloc[splits_2[1]]['Day1DangerAboveTreeline'].head(10)

In [ ]:
#d,t = flatten_check(interp.decoded, interp.targs)
#print(str(skm.accuracy_score(t, d)))
interp = ClassificationInterpretation.from_learner(learn)
conf_matrix = interp.plot_confusion_matrix(normalize=False)
conf_matrix = interp.plot_confusion_matrix(normalize=True)

In [ ]:
learners.keys()


In [ ]:
#top values from model zoo
#ITP1 Trial 289 finished with value: 0.617 and parameters: {'num_days': 28, 'batch_size': 8, 'model': 'InceptionTimePlus', 'itp_num_filters': 32, 'itp_depth': 6, 'itp_dropout_rate': 0.0, 'loss_func': 'LabelSmoothingCrossEntropyFlat', 'learning_rate': 0.0024792334740007253}

#ITP2 Trial 454 finished with value: 0.616 and parameters: {'num_days': 28, 'batch_size': 8, 'model': 'InceptionTimePlus', 'itp_num_filters': 32, 'itp_depth': 9, 'itp_dropout_rate': 0.0, 'loss_func': 'None', 'learning_rate': 0.0024792334740007253}

#ITP4 Trial 161 finished with value: 0.609 and parameters: {'num_days': 84, 'batch_size': 8, 'model': 'InceptionTimePlus', 'itp_num_filters': 32, 'itp_depth': 9, 'itp_dropout_rate': 0.0, 'loss_func': 'None', 'learning_rate': 0.0006723398746247245}

#ITP5 Trial 26 finished with value: 0.611 and parameters: {'num_days': 84, 'batch_size': 8, 'model': 'InceptionTimePlus', 'itp_num_filters': 32, 'itp_depth': 6, 'itp_dropout_rate': 0.2, 'loss_func': 'None', 'learning_rate': 1.339568695002037e-05}

#ITP3 Trial 352 finished with value: 0.604 and parameters: {'num_days': 28, 'batch_size': 64, 'model': 'InceptionTimePlus', 'itp_num_filters': 64, 'itp_depth': 6, 'itp_dropout_rate': 0.0, 'loss_func': 'None', 'learning_rate': 0.0024792334740007253}

#good learning curve
#TCN 1 Trial 268 finished with value: 0.565 and parameters: {'num_days': 28, 'batch_size': 8, 'model': 'TCN', 'TCN_fc_dropout': 0.6000000000000001, 'TCN_conv_dropout': 0.0, 'loss_func': 'LabelSmoothingCrossEntropy', 'learning_rate': 1.339568695002037e-05}
learners = {'ITP1': [], 'ITP2': [], 'ITP3': [], 'ITP4': [], 'ITP5': [], 'TCN1': []}
def get_model_and_lr(model_name, splits=None):
    model = None
    lr = 1e-3
    days = 28
    dls = None
    if k == 'XCM':
        lr = 1e-3
        splits, dls = create_dls(filtered_feature_indexes, days, splits=splits)
        model = XCM(dls.vars, dls.c, dls.len, fc_dropout=0, window_perc=.5, nf=128)
    elif k == 'TST':
        lr = 1e-4
        splits, dls = create_dls(filtered_feature_indexes, days, splits=splits)
        model = TST(dls.vars, dls.c, dls.len, d_ff=512, d_k=64, d_model=128, d_v=64, dropout=.7, fc_dropout=.1, n_heads=16, n_layers=4)
    elif k == 'ITP':
        lr = 1e-3
        splits, dls = create_dls(filtered_feature_indexes, days, splits=splits)
        model = InceptionTimePlus(dls.vars, dls.c, bottleneck=True, coord=False, ks=5, conv_dropout=.8, depth=6, nf=32)
    elif k == 'ITP1':
        lr = 0.0024
        days = 28
        X = X_small_28
        splits, dls = create_dls(filtered_feature_indexes, days, splits=splits)
        model = InceptionTimePlus(dls.vars, dls.c,  conv_dropout=0, depth=6, nf=32)
    elif k == 'ITP2':
        lr = 0.0024
        days = 28
        X = X_small_28
        splits, dls = create_dls(filtered_feature_indexes, days, splits=splits)
        model = InceptionTimePlus(dls.vars, dls.c,  conv_dropout=0, depth=9, nf=32)
    elif k == 'ITP3':
        lr = 0.0024
        days = 28
        X = X_small_28
        splits, dls = create_dls(filtered_feature_indexes, days, splits=splits)
        model = InceptionTimePlus(dls.vars, dls.c,  conv_dropout=0, depth=6, nf=64)
    elif k == 'ITP4':
        lr = 0.00067
        days = 84
        X = X_small_84
        splits, dls = create_dls(filtered_feature_indexes, days, splits=splits)
        model = InceptionTimePlus(dls.vars, dls.c,  conv_dropout=0, depth=9, nf=32)
    elif k == 'ITP5':
        lr = 1e-5
        days = 84
        X = X_small_84
        splits, dls = create_dls(filtered_feature_indexes, days, splits=splits)
        model = InceptionTimePlus(dls.vars, dls.c,  conv_dropout=.2, depth=6, nf=32)
    elif k == 'TCN1':
        lr = 1e-5
        days = 28
        X = X_small_28
        splits, dls = create_dls(filtered_feature_indexes, days, splits=splits)
        model = TCN(dls.vars, dls.c, conv_dropout=0, fc_dropout=.6)
    return model, lr, days, dls, splits

In [ ]:
for k in learners.keys():
    for i in range(5):        
        model, lr, days, dls, splits = get_model_and_lr(k)
        learn = Learner(dls, model, metrics=accuracy, loss_func=LabelSmoothingCrossEntropy(), cbs=[ShowGraphCallback2(), PredictionDynamics(), SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=.001, fname=k + str(i))])
        learn.fit_one_cycle(200, lr_max=lr, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=15))
        learners[k].append(learn)

In [ ]:
del out

In [ ]:
import gc
gc.collect()

In [ ]:
learners.keys()

In [ ]:
for k in learners.keys():
    for i in range(1):
        learners[k][i].export("./models/" + k + str(i) + '.pkl')

In [ ]:
dls.valid

In [ ]:
model = InceptionTimePlus(dls.vars, dls.c, bottleneck=True, coord=False, ks=5, conv_dropout=.8, depth=6, nf=32)

model.load_state_dict(torch.load("./models/" + 'ITP' + '0.pth'))


In [ ]:
learn = Learner(dls, model, metrics=accuracy, loss_func=LabelSmoothingCrossEntropy(), cbs=[ShowGraphCallback2(), SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=.001, fname=k + str(i))])

In [ ]:
learn.get_preds(dl=dls.valid, with_input=False, with_loss=False, with_decoded=True, act=None)

In [ ]:
probas = {}
target = {}
preds = {}

In [ ]:
model, lr, days, dls, splits = get_model_and_lr('ITP1')
for k in learners.keys():    
    probas[k] = [None] * 5
    target[k] = [None] * 5
    preds[k] = [None] * 5
    
    for i in range(5):                
        model, lr, days, dls, splits_new = get_model_and_lr(k, splits=splits)        
        assert splits_new == splits
        model.load_state_dict(torch.load("./models/" + k + str(i) + '.pth'))
        learn = Learner(dls, model, metrics=accuracy, loss_func=LabelSmoothingCrossEntropy(), cbs=[ShowGraphCallback2(), PredictionDynamics(), SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=.001, fname=k + str(i))])
        probas[k][i], target[k][i], preds[k][i] = learn.get_preds(dl=dls.valid, with_input=False, with_loss=False, with_decoded=True, act=None)

In [ ]:
a=torch.zeros(probas['TCN1'][0].shape, dtype=torch.float64, device = 'cuda')
a
#torch.empty_like(a)

In [ ]:
p = torch.zeros(probas['TCN1'][0].shape, dtype=torch.float64, device = 'cpu')

for k in learners.keys():    
    for i in range(5):    
        t = skm.accuracy_score(target[k][i], preds[k][i])
        t2 = skm.accuracy_score(target[k][i], np.argmax(probas[k][i], axis=1))
        if t > .55:
            p += probas[k][i]
        
            print(k + str(i) + str(t) + ' argmax: ' + str(t2))
    print('tmp: ' + str(skm.accuracy_score(np.argmax(p, axis=1), target['ITP1'][i])))
unified=np.argmax(p, axis=1)
print(skm.accuracy_score(unified, target['ITP1'][i]))

In [ ]:
probas[k], target[k], preds[k] = learners[k].get_X_preds(X[splits_2[1]], y[splits_2[1]])

In [ ]:
skm.confusion_matrix(unified, target['ITP1'][i], normalize='true')

In [ ]:
xb, yb = dls.one_batch()

In [ ]:
xb

In [ ]:
xb1 = X[np.where(abs(target-preds) == 0)[0][:64]]
yb1 = y[np.where(abs(target-preds) == 0)[0][:64]]

In [ ]:
xb = TSTensor(xb1).to(torch.device('cuda'))
yb = TensorCategory(yb1).to(torch.device('cuda'))

In [ ]:
detach=True
cpu=True
apply_relu=True
cmap='inferno'
figsize=None
att_maps = get_attribution_map(model, [model.conv2dblock, model.conv1dblock], xb, y=yb, detach=detach, cpu=cpu, apply_relu=apply_relu)
att_maps[0] = ((att_maps[0] - att_maps[0].min()) / (att_maps[0].max() - att_maps[0].min())).mean(dim=0)
att_maps[1] = ((att_maps[1] - att_maps[1].min()) / (att_maps[1].max() - att_maps[1].min())).mean(dim=0)

        

In [ ]:
att_maps[0].shape

In [ ]:
att_maps[0].mean(dim=0).shape

In [ ]:
att_maps[0]

In [ ]:
figsize = ifnone(figsize, (10, 10))
fig = plt.figure(figsize=figsize)
ax = plt.axes()

plt.title('Observed variables')
im = ax.imshow(att_maps[0], cmap=cmap, aspect='auto')
plt.yticks(range(len(filtered_feature_list)), filtered_feature_list)
#plt.yticks(range(len(feature_names)), feature_names)
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
plt.show()

In [ ]:
fig = plt.figure(figsize=figsize)
ax = plt.axes()
plt.title('Time')
im = ax.imshow(att_maps[1], cmap=cmap, aspect='auto')
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
plt.show()

In [ ]:
df = pd.DataFrame(zip(filtered_feature_list, list(att_maps[0].sum(dim=0).numpy()))).sort_values(1, ascending=False)#.plot.barh()#.to_csv('/home/scottcha/Desktop/feature_importance.csv')

In [ ]:
top_45_features = list(df.head(45)[0].values)
top_45_features.sort()

In [ ]:
df2 = pd.concat([df, pd.DataFrame.from_records(df[0].str.split('_', 1))], axis=1)

In [ ]:
df2.columns = ['feature', 'importance', 'var', 'level']
df2.head()

In [ ]:
df2.sort_values(by='importance', ascending=False)[:20].plot.bar(x='feature', y='importance')

In [ ]:
df2.groupby('var').mean('importance').sort_values(by='importance', ascending=False)

In [ ]:
df2.groupby('var').mean('importance').sort_values(by='importance', ascending=False)

In [ ]:
tmp = df.set_index(0)
tmp[tmp[1] > 0]

In [ ]:
#top 30 features
list(df.set_index(0).head(250).index)

In [ ]:
#bottom 30 features
list(df.set_index(0).tail(250).index)

In [ ]:
pd.Series(X[220140][filtered_feature_list.index('TMP_20mb_avg')]).plot()

In [ ]:
from cleanlab.classification import LearningWithNoisyLabels
from sklearn.linear_model import LogisticRegression

In [ ]:
batch_tfms = [TSStandardize(by_var=True), Nan2Value()]
clf = TSClassifier(X, y, splits=splits_2, arch=InceptionTimePlus, batch_size=[64, 128], batch_tfms=batch_tfms, metrics=accuracy, cbs=ShowGraph(), verbose=True, inplace=False)
clf.fit_one_cycle(10, 1e-3)

In [ ]:
test_probas, test_targets, test_preds = clf.get_X_preds(X[splits_2[0]], with_decoded=True)
test_probas, test_targets, test_preds

In [ ]:
splits_2[1]

In [ ]:
from cleanlab.pruning import get_noise_indices

ordered_label_errors = get_noise_indices(
    s=y[splits_2[0]],
    psx=test_probas.numpy(),
    sorted_index_method='normalized_margin', # Orders label errors
 )

In [ ]:
len(ordered_label_errors)

In [ ]:
splits_3=[[x for x in splits_2[0] if x not in ordered_label_errors], splits_2[1]]

In [ ]:
from hyperopt import tpe, hp, fmin

In [ ]:
arch_list = ['InceptionTimePlus']#, 'XceptionTimePlus', 'InceptionTimeXLPlus']#, 'mWDN', 'FCN', 'ResNet', 'xresnet1d34', 'ResCNN', 'LSTM', 'LSTM_FCN']
inception_arch_list = ['InceptionTimePlus', 'InceptionTimePlus17x17', 'InceptionTimePlus32x32', 'InceptionTimePlus47x47', 'InceptionTimePlus62x62', 'InceptionTimeXLPlus']
tst_arch_list = ['TST']

In [ ]:
def objective_xcm(params):
    print(params)
    batch_size, window_perc, loss, fc_dropout, nf = params['batch_size'], params['window_perc'], params['loss'], params['fc_dropout'], params['nf']
    dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[batch_size], batch_tfms=batch_tfms, num_workers=12, inplace=False)
    model = XCM(dls.vars, dls.c, dls.len, window_perc=window_perc, nf=nf, fc_dropout=fc_dropout)
    
    loss_func = None
    if loss == 'LabelSmoothingCrossEntropy':
        loss_func = LabelSmoothingCrossEntropy()
    elif loss == 'LabelSmoothingCrossEntropyFlat':
        loss_func = LabelSmoothingCrossEntropyFlat()
        
    learn = Learner(dls, model, metrics=accuracy, loss_func=loss_func)   
    
    learn.fit_one_cycle(25, lr_max=1e-3, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=3))
    learn.recorder.plot_metrics()
    interp = ClassificationInterpretation.from_learner(learn)
    conf_matrix = interp.plot_confusion_matrix(normalize=True)
    #enable us to use fmin
    #return learn.recorder.values[-1][1]
    return 1.0 - accuracy_local(interp)

In [ ]:
def objective_tst(params):
    print(params)
    d_model, n_heads, d_k, d_v, d_ff, dropout, n_layers, fc_dropout, loss = params['d_model'], params['n_heads'], params['d_k'], params['d_v'], params['d_ff'], params['dropout'], params['n_layers'], params['fc_dropout'], params['loss']
    dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[64], batch_tfms=batch_tfms, num_workers=12, inplace=False)
    model = TST(dls.vars, dls.c, dls.len, d_model=d_model, n_heads=n_heads, d_k=d_k, d_v=d_v, d_ff=d_ff, dropout=dropout, n_layers=n_layers, fc_dropout=fc_dropout)
    
    loss_func = None
    if loss == 'LabelSmoothingCrossEntropy':
        loss_func = LabelSmoothingCrossEntropy()
    elif loss == 'LabelSmoothingCrossEntropyFlat':
        loss_func = LabelSmoothingCrossEntropyFlat()
        
    learn = Learner(dls, model, metrics=accuracy, loss_func=loss_func)   
    
    learn.fit_one_cycle(25, lr_max=1e-4, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=3))
    learn.recorder.plot_metrics()
    interp = ClassificationInterpretation.from_learner(learn)
    conf_matrix = interp.plot_confusion_matrix(normalize=True)
    #enable us to use fmin
    #return learn.recorder.values[-1][1]
    return 1.0 - accuracy_local(interp)

In [ ]:
def objective(params):
    #arch, levels = params['arch'], params['levels']
    print(params)
    arch, ks, bottleneck, coord, dropout, lr, depth, num_filters, loss = params['arch'], params['ks'], params['bottleneck'], params['coord'], params['dropout'], params['lr'], params['depth'], params['num_filters'], params['loss']
    #create the dataloader
    #shouldn't be necessary, but there must be a bug which causes teh tfms to stop being executed on the validation set after a run
    dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[64], batch_tfms=batch_tfms, num_workers=12, inplace=False)

    model = None
    if arch == 'InceptionTime':
        model = InceptionTime(dls.vars, dls.c)
    elif arch == 'InceptionTimePlus':
        model = InceptionTimePlus(dls.vars, dls.c, bottleneck=bottleneck, coord=coord, conv_dropout=dropout, depth=depth, nf=num_filters)
    elif arch == 'InceptionTimePlus17x17':
        model = InceptionTimePlus(dls.vars, dls.c, bottleneck=bottleneck, coord=coord, conv_dropout=dropout, depth=depth, nf=num_filters)
    elif arch == 'InceptionTimePlus32x32':
        model = InceptionTimePlus(dls.vars, dls.c, bottleneck=bottleneck, coord=coord, conv_dropout=dropout, depth=depth, nf=num_filters)
    elif arch == 'InceptionTimePlus47x47':
        model = InceptionTimePlus(dls.vars, dls.c, bottleneck=bottleneck, coord=coord, conv_dropout=dropout, depth=depth, nf=num_filters)
    elif arch == 'InceptionTimePlus62x62':
        model = InceptionTimePlus(dls.vars, dls.c, bottleneck=bottleneck, coord=coord, conv_dropout=dropout, depth=depth, nf=num_filters)
    elif arch == 'InceptionTimeXLPlus':
        model = InceptionTimeXLPlus(dls.vars, dls.c, bottleneck=bottleneck, coord=coord, conv_dropout=dropout, depth=depth, nf=num_filters)
    elif arch == 'XceptionTime':
        model = XceptionTime(dls.vars, dls.c)
    elif arch == 'XceptionTimePlus':
        model = XceptionTimePlus(dls.vars, dls.c,  bottleneck=bottleneck, coord=coord)
    elif arch == 'mWDN':
        model = mWDN(dls.vars, dls.c, seq_len=180, levels=4)
    elif arch == 'FCN':
        model = FCN(dls.vars, dls.c)
    elif arch == 'ResNet':
        model = ResNet(dls.vars, dls.c)
    elif arch == 'xresnet1d34':
        model = xresnet1d34(dls.vars, dls.c)
    elif arch == 'ResCNN':
        model = ResCNN(dls.vars, dls.c)    
    elif arch == 'LSTM':
        model = LSTM(dls.vars, dls.c)
    elif arch == 'LSTM_FCN':
        model = LSTM_FCN(dls.vars, dls.c)    
        
    loss_func = None
    if loss == 'LabelSmoothingCrossEntropy':
        loss_func = LabelSmoothingCrossEntropy()
    elif loss == 'LabelSmoothingCrossEntropyFlat':
        loss_func = LabelSmoothingCrossEntropyFlat()
        
    learn = Learner(dls, model, metrics=accuracy, loss_func=loss_func)   
    #without oversampling
    learn.fit_one_cycle(25, lr_max=lr, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.01, patience=3))
    learn.recorder.plot_metrics()
    interp = ClassificationInterpretation.from_learner(learn)
    conf_matrix = interp.plot_confusion_matrix(normalize=True)
    #enable us to use fmin
    #return learn.recorder.values[-1][1]
    return 1.0 - accuracy_local(interp)

In [ ]:
def objective_lstm(params):
   
    print(params)
    hidden_size, n_layers, bias, rnn_dropout, bidirectional, fc_dropout, plus, lr, loss = params['hidden_size'], params['n_layers'], params['bias'], params['rnn_dropout'], params['bidirectional'], params['fc_dropout'], params['plus'], params['lr'], params['loss']
    #create the dataloader
    #shouldn't be necessary, but there must be a bug which causes teh tfms to stop being executed on the validation set after a run
    dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[32], batch_tfms=batch_tfms, num_workers=12, inplace=False)
    model = None
    if plus:
        model = LSTMPlus(dls.vars, dls.c, hidden_size=hidden_size, n_layers=n_layers, bias=bias, rnn_dropout=rnn_dropout, bidirectional=bidirectional, fc_dropout=fc_dropout)
    else:
        model = LSTM(dls.vars, dls.c, hidden_size=hidden_size, n_layers=n_layers, bias=bias, rnn_dropout=rnn_dropout, bidirectional=bidirectional, fc_dropout=fc_dropout)

    loss_func = None
    if loss == 'LabelSmoothingCrossEntropy':
        loss_func = LabelSmoothingCrossEntropy()
    elif loss == 'LabelSmoothingCrossEntropyFlat':
        loss_func = LabelSmoothingCrossEntropyFlat()
    learn = Learner(dls, model, metrics=accuracy, loss_func=loss_func)   
    #without oversampling
    learn.fit_one_cycle(25, lr_max=lr, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.01, patience=3))
    learn.recorder.plot_metrics()
    interp = ClassificationInterpretation.from_learner(learn)
    conf_matrix = interp.plot_confusion_matrix(normalize=True)
    #enable us to use fmin
    return 1.0 - accuracy_local(interp)

In [ ]:
space = {
    'arch': hp.choice('arch', arch_list),
    'ks': hp.choice('ks', [5, 10, 40, 60, 80]),
    'bottleneck': hp.choice('bottleneck', [True]),
    'coord': hp.choice('coord', [False]),
    #'stride': hp.choice('stride', [1,3,5,7,9]),
    'dropout': hp.choice('dropout', [0., .2, .5, .8]),
    'lr': hp.choice('lr', [1e-2, 1e-3, 1e-4]),
    'depth': hp.choice('depth', [3, 6, 9]),
    'num_filters': hp.choice('num_filters', [32, 64, 96]),
    'loss': hp.choice('loss', ['LabelSmoothingCrossEntropy', 'LabelSmoothingCrossEntropyFlat', 'None'])
}

space_lstm = {
    'hidden_size': hp.choice('hidden_size', [100, 150, 200, 250]),
    'n_layers': hp.choice('n_layers', [1,5,8,10]),
    'bias': hp.choice('bias', [True, False]),
    'rnn_dropout': hp.choice('rnn_dropout', [0, .2, .4, .6, .8]),
    'bidirectional': hp.choice('bidirectional', [True, False]),
    'fc_dropout': hp.choice('fc_dropout', [0, .2, .4, .6, .8]),
    'plus': hp.choice('plus', [True, False]),
    'lr': hp.choice('lr', [1e-2, 1e-3, 1e-4]),
    'loss': hp.choice('loss', ['LabelSmoothingCrossEntropy', 'LabelSmoothingCrossEntropyFlat', 'None'])
}

space_tst = {
    'd_model': hp.choice('d_model', [64, 128, 256]),
    'n_heads': hp.choice('n_heads', [8, 16, 24]), 
    'd_k': hp.choice('d_k', [16, 32, 64, None]), 
    'd_v': hp.choice('d_v', [16, 32, 64, None]), 
    'd_ff': hp.choice('d_ff', [128, 256, 512]), 
    'dropout': hp.choice('dropout', [0., .1, .5, .7]) ,
    'n_layers': hp.choice('n_layers', [2,3,4,5]), 
    'fc_dropout': hp.choice('fc_dropout', [0, .1, .5, .7]),
    'loss': hp.choice('loss', ['LabelSmoothingCrossEntropy', 'LabelSmoothingCrossEntropyFlat', 'None'])
}

space_xcm = {
    'batch_size': hp.choice('batch_size', [1, 16, 32, 64]),
    'window_perc': hp.choice('window_perc', [.3, .5, .7, 1]),
    'nf': hp.choice('nf', [64, 128, 256]),
    'fc_dropout': hp.choice('fc_dropout', [0, .1, .5, .8]),
    'loss': hp.choice('loss', ['LabelSmoothingCrossEntropy', 'LabelSmoothingCrossEntropyFlat', 'None'])
}


In [ ]:
best = fmin(
        fn=objective_xcm,
        space=space_xcm,
        algo=tpe.suggest,
        max_evals=25
)
print(best)

In [ ]:
best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=50
)
print(best)

In [ ]:
best

In [ ]:
#large
best = fmin(
        fn=objective_lstm,
        space=space_lstm,
        algo=tpe.suggest,
        max_evals=25
)

In [ ]:
best = fmin(
        fn=objective_tst,
        space=space_tst,
        algo=tpe.suggest,
        max_evals=50
)

In [ ]:
#optimum for 30k samples {'arch': 'InceptionTimePlus', 'bottleneck': True, 'coord': False, 'depth': 3, 'dropout': 0.0, 'ks': 60, 'lr': 0.001, 'num_filters': 64}
#for 180k samples 72 features {'arch': 'InceptionTimePlus', 'bottleneck': True, 'coord': False, 'depth': 6, 'dropout': 0.8, 'ks': 5, 'loss': 'LabelSmoothingCrossEntropy', 'lr': 0.01, 'num_filters': 32}
#{'arch': 'InceptionTimePlus', 'bottleneck': True, 'coord': False, 'depth': 6, 'dropout': 0.8, 'ks': 5, 'loss': 'LabelSmoothingCrossEntropy', 'lr': 0.001, 'num_filters': 64}
dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[64], batch_tfms=batch_tfms, num_workers=12, inplace=False)
model = InceptionTimePlus(dls.vars, dls.c, bottleneck=True, coord=False, ks=5, conv_dropout=.8, depth=6, nf=32)
learn = Learner(dls, model, loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy)   

In [ ]:
learn.lr_find()

In [ ]:

learn.fit_one_cycle(25, lr_max=1e-3, cbs=[EarlyStoppingCallback(monitor='accuracy', min_delta=1e-2, patience=3), ShowGraphCallback2()])#, SaveModelCallback(every_epoch=True)])


In [ ]:
#TST 30 days, 200k samples, 74 features
learn.fit_one_cycle(25, lr_max=1e-4, cbs=[EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=5)])#, SaveModelCallback(every_epoch=True)])


In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
conf_matrix = interp.plot_confusion_matrix(normalize=True)
conf_matrix = interp.plot_confusion_matrix(normalize=False)

In [ ]:
accuracy_local(interp)

In [ ]:
models = []
for i in range(5):
    model = None
    dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[64], batch_tfms=batch_tfms, num_workers=12, inplace=False)
    model = TST(dls.vars, dls.c, dls.len, d_ff=512, d_k=64, d_model=128, d_v=64, dropout=.7, fc_dropout=.1, n_heads=16, n_layers=4)
    learn = Learner(dls, model, loss_func=LabelSmoothingCrossEntropyFlat(), metrics=accuracy,  cbs=ShowGraphCallback2())
    learn.fit_one_cycle(25, lr_max=1e-4, cbs=[EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=3)])
    interp = ClassificationInterpretation.from_learner(learn)
    conf_matrix = interp.plot_confusion_matrix(normalize=True)
    print(str(accuracy_local(interp)))
    models.append(model)
    

In [ ]:
y[splits_2[1]]

In [ ]:
ys = []
ys[0] = models[0]

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
conf_matrix = interp.plot_confusion_matrix(normalize=True)

In [ ]:
#30 days, 50% samples , 74 features
learn.fit_one_cycle(25, lr_max=1e-4, cbs=[EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=5)])#, SaveModelCallback(every_epoch=True)])


In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
conf_matrix = interp.plot_confusion_matrix(normalize=True)

In [ ]:
X_valid = learn.dls.valid.dataset[:100][0].data
y_valid = learn.dls.valid.dataset[:100][1].data.numpy()

In [ ]:
X_valid.dtype

In [ ]:
sys.getsizeof(X_valid)

In [ ]:
learn.dls.valid.splits

In [ ]:
X_valid.shape

In [ ]:
y_valid.numpy()

In [ ]:
metrics = [mn for mn in learn.recorder.metric_names if mn not in ['epoch', 'train_loss', 'valid_loss', 'time']]

In [ ]:
key_metric_idx=0
if len(metrics) == 0 or key_metric_idx is None:
    metric_name = learner.loss_func.__class__.__name__
    key_metric_idx = None
else:
    metric_name = metrics[key_metric_idx]
    metric = learn.recorder.metrics[key_metric_idx].func
print(f'Selected metric: {metric_name}')

In [ ]:
feature_names = pd.read_csv(ml_path + '/FeatureLabels_' + file_label + '.csv')
feature_names = feature_names['0'].sort_values().values

In [ ]:
if feature_names is None:
    feature_names = [f"var_{i}" for i in range(X_valid.shape[1])]

In [ ]:
results = []
print('Computing feature importance...')

COLS = ['BASELINE'] + list(feature_names)

In [ ]:
len(dls.valid.dataset.tls[0]._splits)

In [ ]:
dls.valid.dataset[0:10][0].data.shape

In [ ]:
X_valid[:, k-1].clone()

In [ ]:
k = 5
random_state=23
if k>0:
    save_feat = X_valid[:, k-1].clone()
    X_valid[:, k-1] = random_shuffle(X_valid[:, k-1].flatten(), random_state=random_state).reshape(X_valid[:, k-1].shape)
if key_metric_idx is None:
    value = learn.get_X_preds(X_valid, y_valid, with_loss=True)[-1].mean().item()
else:
    output = learn.get_X_preds(X_valid, y_valid)
    value = metric(output[0], output[1]).item()
    print(f"{k:3} feature: {COLS[k]:20} {metric_name}: {value:8.6f}")
    results.append([COLS[k], value])
if k>0:
    X_valid[:, k-1] = save_feat
    del save_feat; gc.collect()

In [ ]:
??learn.dls.valid.new_dl

In [ ]:
from __future__ import print_function
from sys import getsizeof, stderr
from itertools import chain
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass

def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}

    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o), file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break
        return s

    return sizeof(o)



In [ ]:
del dl; gc.collect()

In [ ]:
bs=64
with_input=False
with_decoded=True
with_loss=False
dl = learn.dls.valid.new_dl(X_valid, y=y_valid)
if bs: setattr(dl, "bs", bs)
else: assert dl.bs, "you need to pass a bs != 0"
output = list(learn.get_preds(dl=dl, with_input=with_input, with_decoded=with_decoded, with_loss=with_loss, reorder=False))
if with_decoded and len(learn.dls.tls) >= 2 and hasattr(learn.dls.tls[-1], "tfms") and hasattr(learn.dls.tls[-1].tfms, "decodes"):
    output[2 + with_input] = learn.dls.tls[-1].tfms.decode(output[2 + with_input])


In [ ]:
learn.dls.tls[-1]

In [ ]:
output = learn.get_X_preds(X_valid, y_valid)

In [ ]:
import sys

In [ ]:
total_size(learn)

In [ ]:
total_size(dl)

In [ ]:
total_size(output)

In [ ]:
total_size(X_valid)

In [ ]:
total_size(results)

In [ ]:
total_size(value)

In [ ]:
results

In [ ]:
gc.collect()

In [ ]:
b = dls.valid.one_batch()



In [ ]:
dsets.valid

In [ ]:
probas, target, preds  = learn.get_preds(dl=dls.valid, with_input=False, with_loss=False, with_decoded=True, act=None)

In [ ]:
X[splits_2[1]].shape

In [ ]:
dls.bs

In [ ]:
learn.feature_importance(X[splits_2[1]], y[splits_2[1]])

In [ ]:
for i in range(75):
    print(str(i))
    probas, target, preds = learn.get_X_preds(X[splits_2[1]], y[splits_2[1]])

In [ ]:
import gc
gc.collect()

In [ ]:
#60000 samples 0 dropout
learn.fit_one_cycle(25, lr_max=1e-4, cbs=[EarlyStoppingCallback(monitor='accuracy', min_delta=0.01, patience=5)])#, SaveModelCallback(every_epoch=True)])
learn.recorder.plot_metrics()
interp = ClassificationInterpretation.from_learner(learn)
#conf_matrix = interp.plot_confusion_matrix(normalize=True)

print(accuracy_local(interp))

In [ ]:
conf_matrix = interp.plot_confusion_matrix(normalize=True)

In [ ]:
#60000 samples
learn.fit_one_cycle(25, lr_max=1e-3, cbs=[EarlyStoppingCallback(monitor='accuracy', min_delta=0.01, patience=3)])#, SaveModelCallback(every_epoch=True)])
learn.recorder.plot_metrics()
interp = ClassificationInterpretation.from_learner(learn)
conf_matrix = interp.plot_confusion_matrix(normalize=True)

print(accuracy_local(interp))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
learn.fit_one_cycle(5, lr_max=1e-2)#, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.1, patience=5))
learn.recorder.plot_metrics()
interp = ClassificationInterpretation.from_learner(learn)
conf_matrix = interp.plot_confusion_matrix(normalize=True)

print(accuracy_local(interp))

In [ ]:
ordered_label_errors.shape

In [ ]:
[[x for x in splits_2[0] if x not in ordered_label_errors], splits_2[1]]

In [ ]:
batch_tfms = [TSStandardize(by_var=True), Nan2Value()]
clf = TSClassifier(X, y, splits=[[x for x in splits_2[0] if x not in ordered_label_errors], splits_2[1]], arch=InceptionTimePlus, batch_tfms=batch_tfms, metrics=accuracy, cbs=ShowGraph(), verbose=True)
clf.fit_one_cycle(3, 1e-2)

In [ ]:
interp = ClassificationInterpretation.from_learner(clf)
conf_matrix = interp.plot_confusion_matrix(normalize=True)

In [ ]:
feature_names = pd.read_csv(ml_path + '/FeatureLabels_' + file_label + '.csv')
feature_names = feature_names['0'].sort_values().values

In [ ]:
feature_names.shape

In [ ]:
X.shape

In [ ]:
import xarray as xr

In [ ]:
da = xr.DataArray(X, dims=['sample','feature', 'timestep'], coords=[range(0,12000),feature_names,range(0,180)])

In [ ]:
feature_names

In [ ]:
filtered_feature_indexes

In [ ]:
avg_features = [x for x in feature_names if 'avg' in x ]

In [ ]:
many_level_features = ['ABSV', 'HGT']

In [ ]:
filtered_features

In [ ]:
feature = ['WEASD_surface_max', 'SNOD_surface_max', 'UGRD_30M0mbaboveground_max', 'VGRD_30M0mbaboveground_max', 'TMP_surface_min', 'HGT_surface_min', 'VVEL_150mb_avg', 'APCP_surface_sum', 'ACPCP_surface_sum', 'WATR_surface_sum']
small_feature_indexes = [list(feature_names).index(x) for x in feature]

In [ ]:
da.sel(feature='SNOD_surface_avg', sample=4).plot()

In [ ]:
da.sel(feature='APCP_surface_sum', sample=range(0,100)).plot()

In [ ]:
y.shape


In [ ]:
#X, y, splits = get_classification_data('ECG200', split_data=False)

In [ ]:
X.shape

In [ ]:
da.sel(feature=feature, timestep=timestep).values.reshape(6500,len(feature),len(timestep))[splits_2[0][8]][7]

In [ ]:
timestep=range(0,180)
feature = ['WEASD_surface_max', 'SNOD_surface_max', 'UGRD_30M0mbaboveground_max', 'VGRD_30M0mbaboveground_max', 'TMP_surface_min', 'HGT_surface_min', 'VVEL_150mb_avg', 'APCP_surface_sum', 'ACPCP_surface_sum', 'WATR_surface_sum']
bad_features = ['PRES_surface_max', 'TSOIL_1M2mbelowground_max', 'TMP_surface_min', 'RH_2maboveground_max']
tfms = [None, [Categorize()]]
X = da.sel(feature=feature, timestep=timestep).values.reshape(12000,len(feature),len(timestep))
dsets2 = TSDatasets(X, y, tfms=tfms, splits=splits_2, inplace=True)
#create the dataloader
dls2 = TSDataLoaders.from_dsets(dsets2.train, dsets2.valid, bs=[64], batch_tfms=[TSStandardize(by_var=True), Nan2Value()], num_workers=0)

In [ ]:
ts_features = get_ts_features(np.nan_to_num(X), y, n_jobs=20)

In [ ]:
ts_features.shape

In [ ]:
dls2.train.show_batch()

In [ ]:
dls2.valid.show_batch()

In [ ]:
??get_ts_features

In [ ]:
#TST: 30: .29
#TST: 80: .2
#TST: 180: .4

#Try TST
model = InceptionTimePlus(dls2.vars, dls2.c, dls2.len)#, res_dropout=.3, fc_dropout=.8)
#config = {'res_dropout':.3, 'fc_dropout':.8}
learn = Learner(dls2, model, loss_func=LabelSmoothingCrossEntropy(), metrics=[RocAuc(), accuracy],  cbs=ShowGraphCallback2())
#learn = TSClassifier( y, splits=splits_3, arch=TST, arch_config=config, batch_tfms=[TSStandardize(), Nan2Value()], metrics=accuracy, cbs=ShowGraph(), verbose=True)
#learn.fit_one_cycle(10, 1e-4)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(10, 1e-4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
conf_matrix = interp.plot_confusion_matrix(normalize=True)

print(accuracy_local(interp))


In [ ]:
from captum.attr import IntegratedGradients
from captum.attr import LayerConductance
from captum.attr import NeuronConductance

In [ ]:
net = learn.model.cpu()

In [ ]:
b = dls.one_batch()

In [ ]:
test_input_tensor = b[0].type(torch.FloatTensor)

In [ ]:
test_labels = b[1].cpu().data.numpy()

In [ ]:
out_probs = net(test_input_tensor).detach().cpu().numpy()
out_classes = np.argmax(out_probs, axis=1)
train_accuracy = sum(out_classes == test_labels) / len(test_labels)
print("Accuracy:", train_accuracy )

In [ ]:
mlflow.log_metric('Train Accuracy', train_accuracy)

In [ ]:
ig = IntegratedGradients(net)

In [ ]:
#test_input_tensor.requires_grad_()
attr, delta = ig.attribute(test_input_tensor,target=1, return_convergence_delta=True)
attr = attr.detach().numpy()

In [ ]:
feature_names = feature_names[feature_filter]

In [ ]:
# Helper method to print importances and visualize distribution
def visualize_importances(feature_names, importances, title="Average Feature Importances", plot=True, axis_title="Features"):
    pd.DataFrame(zip(feature_names, np.mean(importances, axis=(0,2)))).sort_values(1, ascending=False)[:25].plot.barh(x=0, y=1)

visualize_importances(feature_names, attr)

In [ ]:
top_features =  pd.DataFrame(zip(feature_names, np.mean(attr, axis=(0,2))), columns=['Feature', 'Importance']).sort_values('Importance', ascending=False)

In [ ]:
top_features.to_csv('../Scratch/Notebooks/' + '/FeatureImportances.csv')

In [ ]:
top_features = pd.read_csv('../Scratch/Notebooks/' + '/FeatureImportances.csv')

In [ ]:
orig_X = X

In [ ]:
orig_feature_means = feature_means

In [ ]:
orig_feature_std = feature_std

In [ ]:
mlflow.end_run()

In [ ]:
feature_filter = top_features[:100].index

In [ ]:
X = orig_X[:, feature_filter, :]
feature_means = orig_feature_means[feature_filter]
feature_std = orig_feature_std[feature_filter]

In [ ]:
fill_values = torch.zeros_like(TSTensor(X))
for i in range(0,X.shape[1]):
    fill_values[:,i,:] = torch.full_like(TSTensor(X[:,i,:]), feature_means[i])
        
X_noNan = torch.where(torch.isnan(TSTensor(X)), fill_values, TSTensor(X))

In [ ]:
X_noNan[:,:,-30:].shape